In [1]:
import uuid
import pandas as pd
from tqdm import tqdm

In [2]:
orders_df = pd.read_csv('../data/orders-1.csv')

In [ ]:
# just to adhere to my definition of the OrdrerRequest dataclass
# unix timestamp is more efficient for computation and storage
# trader_uuid is more secure and flexible than group number
# (in real life, traders won't identify themselves with group numbers)

orders_df["unix_timestamp"] = pd.to_datetime(orders_df["Timestamp"]).astype(int) // 10**9
group_to_uuid = {group: uuid.uuid4() for group in orders_df["Group Number"].unique()}
orders_df["trader_uuid"] = orders_df["Group Number"].map(group_to_uuid)

In [5]:
import asyncio
from order_book_simulator.config import OrderRequest, Priority
from order_book_simulator.config import OrderRequest, Side, Priority
from order_book_simulator.core import OrderBook, MatchingEngine
from order_book_simulator.websocket import ConnectionManager
from order_book_simulator.analysis import MetricsCalculator

In [7]:
order_book = OrderBook()
metrics_calculator = MetricsCalculator(order_book, 0.01)
connection_manager = ConnectionManager()
matching_engine = MatchingEngine(order_book, 
                                 connection_manager, 
                                 asyncio.get_event_loop(), 
                                 metrics_calculator)

In [ ]:
filled_trades = []
total_orders = len(orders_df)
checkpoints = {4, 19, total_orders - 1}  # 5th, 20th, and last order (0-indexed)

for index, row in tqdm(orders_df.iterrows(), total=total_orders, desc="Processing orders"):
    order_request = OrderRequest(
        trader_id = row["trader_uuid"],
        side = row["Buy Sell Direction"],
        priority=Priority.HIGH,
        timestamp=row["unix_timestamp"],
        price=row["Order Price"],
        quantity=row["Quantity"]
    )
    trade = matching_engine.process_order(order_request)
    if trade:
        filled_trades.extend(trade)
    
    # Print order book at checkpoints
    if index in checkpoints:
        label = "FINAL" if index == total_orders - 1 else f"#{index + 1}"
        print(f"\n=== Order Book after {label} orders ===")
        print(order_book)

Processing orders:   0%|          | 0/248 [00:00<?, ?it/s]


=== Order Book after 5 orders ===


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     11.00 │          100 │     12.00 │           30 │
│           │              │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘



=== Order Book after 20 orders ===


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     11.00 │           50 │     12.00 │           35 │
│     10.00 │          135 │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘



=== Order Book after 248 orders (FINAL) ===


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     10.00 │          190 │     11.00 │           27 │
│      9.00 │          228 │     12.00 │          484 │
│      8.00 │           25 │     13.00 │          155 │
│      7.00 │          130 │     15.00 │           50 │
│           │              │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘

Processing orders: 100%|██████████| 248/248 [00:00<00:00, 4060.00it/s]

In [10]:
metrics_calculator.calculate_averages()

--- Final Metrics ---
Average Bid-Ask Spread (in Ticks): 122.6054
Average Bid-Ask Spread (in Basis Points): 1223.5514
Average Best-Level Market Depth (in Dollars): $2,823.51


{'avg_spread_ticks': np.float64(122.60536398467433),
 'avg_spread_bps': np.float64(1223.551382501319),
 'avg_depth_dollars': np.float64(2823.5057471264367)}